In [ ]:
from pathlib import Path
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)


DATA_PATH = Path.cwd().parent / "data" if Path.cwd().name == "src" else Path.cwd() / "data"
DATA_PATH

In [ ]:
from tensorflow.config import list_physical_devices
print("Num GPUs Available: ", len(list_physical_devices('GPU')))

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices()[1])

tf.config.experimental.set_memory_growth(tf.config.list_physical_devices()[1], True)

In [ ]:
image_size = (256, 256)
batch_size = 64
# Data from https://www.kaggle.com/competitions/dogs-vs-cats/
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_PATH / "base_input" / "train",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_PATH / "base_input" / "valid",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_PATH / "nobg_input" / "test",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

In [ ]:
input_shape = (256, 256, 3)
num_classes = 101

In [ ]:
def one_hot_encode(image, label):
    label = tf.one_hot(label, depth=num_classes)
    return image, label

train_ds = train_ds.map(one_hot_encode)
val_ds = val_ds.map(one_hot_encode)

In [ ]:
from tensorflow.keras.applications import EfficientNetB2
from keras.models import Model

model = EfficientNetB2(include_top=False, input_shape=input_shape)

for layer in model.layers:
    layer.trainable = False
    
flat = layers.Flatten()(model.layers[-1].output)
drop1 = layers.Dropout(0.5)(flat)
cls = layers.Dense(512, activation='relu')(drop1)
drop2 = layers.Dropout(0.5)(cls)
output = layers.Dense(num_classes, activation='softmax')(drop2)
model = Model(inputs=model.inputs, outputs=output)

In [ ]:
epochs = 100
checkpoint_filepath = Path().cwd() / "model_ckpts" / "base_model.weights.h5"

if checkpoint_filepath.exists():
    model.load_weights(checkpoint_filepath)

model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=0.0002), metrics=["accuracy"])

callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss', 
        patience=5
    ),
    keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1
    )
]

model.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=callbacks)